In [13]:
from Bio import AlignIO
#Alignment conducted with multiple alignment tool @ http://www.ebi.ac.uk/Tools/msa/clustalo/
aln = AlignIO.read('cpec-align.clustal', 'clustal')
print(aln[:, -1000:-200])

SingleLetterAlphabet() alignment with 8 rows and 800 columns
NGNNNNTAANTTNTNNNNNCN--CTNNNNNNNANAGTGGGAGAT...TAA vic-pb2-cpec-L16-f-vic-EM28_G11.ab1
NGNNNNTAAN-TTANNNNTCNTN-NNNNNNNNNNNNNGGGAGAT...TAA vic-pb2-cpec-L11-f-vic-EM28_F11.ab1
NNACAATAACTTANNNNNNCNNNCNNNNN-NNNNNNNNGGAGAT...TAA vic-pb2-cpec-L1-f-vic-EM28_A11.ab1
NGANANNNNNNTNTNNNNTCGNNCNNNANGNNNNNNNNGGAGAT...TAA vic-pb2-cpec-L6-f-vic-EM28_D11.ab1
NGNCNNTAANNTNNNNNNNCGNNCNGNANGNNNNNN-NNGAGAT...TAA vic-pb2-cpec-L3-f-vic-EM28_B11.ab1
NGNCNNTAANNTNTNNNNNCGCNCNNNANNN-NNNN-NGGAGAT...TAA vic-pb2-cpec-L4-f-vic-EM28_C11.ab1
TGACAATAACTTAT----TCA--TCGTCGATGA-TGTGGGAGAT...--- A/Victoria/361/2011|KJ942687
NGACAATTACTTAT----TNN--TCGTCGANGANNGNGGGAGAT...--- vic-pb2-cpec-L10-f-vic-EM28_E11.ab1


In [117]:
L16 = aln[0]
L11 = aln[1]
L1 = aln[2]
L6 = aln[3]
L3 = aln[4]
L4 = aln[5]

L10 = aln[7]
vic = aln[6]


str(L16.seq)

'---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [115]:

def is_good(index, sequence):
    good = 0
    for next_base in sequence[index+1:index+200]: #200 chosen as large enough to ensure 
        if next_base != "-":
            good += 1      
    if good > 198:
        return True
def find_end(index, sequence):
    for more_index, more_base in enumerate(sequence[index:]):
        if more_base == "-":
            return more_index
    return False
            


start_list = []
end_list = []

for sequence in aln[0:6]:
    for index, base in enumerate(sequence):
        if base != "-":
            if is_good(index, sequence):
 
                start_list.append(index)
                if find_end(index, sequence) != False:
                    end_list.append(index + find_end(index, sequence))

                break

from collections import Counter
data = Counter(end_list)
end = 1894
#end = data.most_common(1)[0][0] 

start = max(start_list)       

print(start, end)      
print(start_list, end_list)

# trim_seq = []
# for sequence in aln[0:6]:
#     sequence = sequence[start:end]
#     trim_seq.append(sequence)
    
def is_diff(mutant, vic = aln[6]):
    num_mut = 0
    where_mut = []
    for base_index, base1 in enumerate(mutant[start:end]):
        if base1 != vic[base_index]:
            num_mut += 1
            where_mut.append(base_index)

    return num_mut
mut_dict = dict([])
seq_num = 0
for sequence in aln[0:6]:
    mut_dict[seq_num] = is_diff(sequence)
    seq_num += 1
print(L16[start:end])
print(mut_dict)
print(aln[0][start:end])
list_vic = list(vic)
vic_L16 = list_vic.copy()
vic_L16[start:end] = list(aln[0])[start:end]

#print(str(L16))
                
# from difflib import Differ

# def is_diff(seq1, seq2)
#     d = Differ()
#     list(compare(seq1, seq2))

    


1677 1894
[1629, 1630, 1636, 1677, 1643, 1643] [1894, 2122, 2122, 2122, 2122, 2122]
ID: vic-pb2-cpec-L16-f-vic-EM28_G11.ab1
Name: <unknown name>
Description: vic-pb2-cpec-L16-f-vic-EM28_G11.ab1
Number of features: 0
Seq('TACCTATCAATGGATCATCAGAAATTGGGAAGCTGTCAAAATTCAATGGTCTCA...CCG', SingleLetterAlphabet())
{0: 162, 1: 161, 2: 161, 3: 160, 4: 162, 5: 161}
ID: vic-pb2-cpec-L16-f-vic-EM28_G11.ab1
Name: <unknown name>
Description: vic-pb2-cpec-L16-f-vic-EM28_G11.ab1
Number of features: 0
Seq('TACCTATCAATGGATCATCAGAAATTGGGAAGCTGTCAAAATTCAATGGTCTCA...CCG', SingleLetterAlphabet())


In [101]:
len("CGTCTTGAGAGGGTTCCTCATT-ATAGGTAAGGAAGACAGGAGATATGGACCAGCATTGA")

60

In [ ]:
300